In [1]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

### 2016 2C 1 Parcial

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [10]:
documents_raw = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores_raw = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30),
]

In [11]:
documents = sc.parallelize(documents_raw)
scores = sc.parallelize(scores_raw)

In [12]:
terms = documents.flatMap(lambda x: [(word, x[0]) for word in x[1].split()])

In [13]:
scores_by_word = scores.map(lambda x: (x[1], (x[0], x[2])))

In [14]:
total = terms.join(scores_by_word)
total.collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30)))]

In [15]:
by_user = total.map(lambda x: ((x[1][1][0], x[1][0]), x[1][1][1])).reduceByKey(lambda x, y: x + y).cache()
by_user.first()

(('thom', 1), 2)

In [16]:
users = scores.map(lambda x: x[0]).distinct() # Usuarios unicos
docs = documents.map(lambda x: x[0])
users_docs = users.cartesian(docs).map(lambda x: ((x[0],x[1]), 0)).cache()
users_docs.collect()

[(('thom', 1), 0),
 (('thom', 2), 0),
 (('thom', 3), 0),
 (('thom', 4), 0),
 (('thom', 5), 0),
 (('thom', 6), 0),
 (('thom', 7), 0),
 (('thom', 8), 0),
 (('thom', 9), 0),
 (('martin', 1), 0),
 (('martin', 2), 0),
 (('martin', 3), 0),
 (('martin', 4), 0),
 (('martin', 5), 0),
 (('martin', 6), 0),
 (('martin', 7), 0),
 (('martin', 8), 0),
 (('martin', 9), 0)]

In [17]:
by_user.rightOuterJoin(users_docs).map(lambda x: (x[0][0], x[0][1], 0 if x[1][0] is None else x[1][0])).collect()

[('thom', 1, 2),
 ('thom', 2, 0),
 ('thom', 3, 0),
 ('thom', 5, 0),
 ('thom', 4, 0),
 ('thom', 6, 0),
 ('thom', 7, 0),
 ('thom', 9, 0),
 ('thom', 8, 0),
 ('martin', 9, 0),
 ('martin', 8, 0),
 ('martin', 5, 0),
 ('martin', 4, 0),
 ('martin', 6, 0),
 ('martin', 7, 0),
 ('martin', 1, -2),
 ('martin', 2, 0),
 ('martin', 3, 60)]

### 2017 1C 1 Parcial

Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

In [18]:
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

stats = sc.parallelize(players_all_time_stats)
scores = sc.parallelize(scores)

In [19]:
stats = stats.map(lambda x: (x[0], (x[1],x[2])))
stats.first()

(1, ('Manu Ginobili', 800))

In [20]:
scores_by_match = scores.map(lambda x: ((x[0], x[1]), x[3])).reduceByKey(lambda x, y: x + y)\
                                  .map(lambda x: (x[0][0], x[1]))
scores_by_match.first()

(1, 300)

In [21]:
resul = scores_by_match.join(stats).filter(lambda x: x[1][0] > x[1][1][1]).map(lambda x: (x[1][1][0])).distinct()

In [22]:
resul.collect()

['Kobe Bryant', 'Marc Gasol']